In [73]:
from Bio.PDB import PDBParser
import string
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

# create parser
parser = PDBParser()
alphabet_str= string.ascii_uppercase
alphabet_list=list(alphabet_str)

# read structure of ligand from file
structure = parser.get_structure('LIGAND','../originals/ligand-aa.pdb')
model_ligand = structure[0]
for i in list(string.ascii_uppercase):
    try:
        chain_ligand = model_ligand[i]
    except KeyError:
        continue

# read structure of receptor from file
structure = parser.get_structure('RECEPTOR', '../originals/receptor-aa.pdb')
model_receptor = structure[0]
for i in list(string.ascii_uppercase):
    try:
        chain_receptor = model_receptor[i]
    except KeyError:
        continue



### find distance between residue of ligand and all residues of receptor
close_residues=[]
for residue1 in chain_ligand:
    #print(residue1)
    for residue2 in chain_receptor:
        if residue1 != residue2:
            # compute distance between CA atoms
            try:
                distance = residue1['CA'] - residue2['CA']
            except KeyError:
                ## no CA atom, e.g. for H_NAG
                continue
            ### find residues with distance shorter than given threashold (here 7 Angstrom)
            if distance < 7:
                #print(residue1, residue2, distance)
                if residue1.get_full_id()[3][1] not in close_residues:
                    close_residues.append(residue1.get_full_id()[3][1])
                #print(residue1.get_full_id()[3][1])
print(close_residues)

<Residue GLY het=  resseq=33 icode= > <Residue SER het=  resseq=213 icode= > 6.863723
33
<Residue GLY het=  resseq=33 icode= > <Residue SER het=  resseq=214 icode= > 5.305806
33
<Residue SER het=  resseq=34 icode= > <Residue SER het=  resseq=214 icode= > 6.869789
34
<Residue PRO het=  resseq=35 icode= > <Residue TRP het=  resseq=211 icode= > 6.977177
35
<Residue PRO het=  resseq=35 icode= > <Residue GLY het=  resseq=212 icode= > 6.819282
35
<Residue PRO het=  resseq=35 icode= > <Residue SER het=  resseq=213 icode= > 6.613491
35
<Residue PRO het=  resseq=35 icode= > <Residue SER het=  resseq=214 icode= > 5.431335
35
<Residue VAL het=  resseq=36 icode= > <Residue TRP het=  resseq=211 icode= > 5.206075
36
<Residue VAL het=  resseq=36 icode= > <Residue GLY het=  resseq=212 icode= > 3.8979273
36
<Residue VAL het=  resseq=36 icode= > <Residue SER het=  resseq=213 icode= > 4.1230845
36
<Residue VAL het=  resseq=36 icode= > <Residue SER het=  resseq=214 icode= > 3.8180435
36
<Residue THR het= 

IndexError: list index out of range

In [67]:
### get dictionary with flexible residues in loop clusters:
def grouper(iterable, threshold):
    prev = None
    group = []
    for item in iterable:
        if not prev or item - prev <= threshold:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group
flex_loops_dict=dict(enumerate(grouper(close_residues, 2), 1))
print(flex_loops_dict)

{1: [33, 34, 35, 36, 37, 38, 40, 41], 2: [63]}


In [81]:
### get first and last residue of loops as cut positions:
cut_pos=[]
for i in flex_loops_dict:
    if len(flex_loops_dict[i])>1:
        cut_pos.append([flex_loops_dict[i][0]-1,flex_loops_dict[i][-1]+1])
for i in range(len(cut_pos)):
    cut1=cut_pos[i][0]
    cut2=cut_pos[i][1]
print(cut1,cut2)

32 42
